In [ ]:
#Tensorflow
!pip3 install tensorflow==2.4.0

#Other dependencies
!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0

#ImageAI
!pip install imageai --upgrade

     |█████████████████████████▋      | 315.9 MB 1.4 MB/s eta 0:00:58ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 128, in resolve
    requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 473, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install detecto

In [ ]:
#import libraries

import numpy as np
import cv2
import matplotlib.pyplot as plt
from math import sqrt
import sys
sys.setrecursionlimit(7000)
import cv2 as cv
import matplotlib

from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid
from torchvision import transforms

from shapely.geometry import Point, Polygon

import warnings
warnings.filterwarnings('ignore')

In [ ]:
cd /content/drive/MyDrive/OxfordTown/Social-Distance-Surveillance-master/train-validation-torch

/content/drive/MyDrive/OxfordTown/Social-Distance-Surveillance-master/train-validation-torch


In [ ]:
#function to add padding to the image
def padding(image,n=100):
    
    #inversion of colours as cv2 takes BGR format by default
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #add padding(border) to images
    constant= cv.copyMakeBorder(image,1*n,1*n,1*n,1*n,cv.BORDER_CONSTANT) 
    
    return constant


#function to rotate image to a particular
def rotate(img, angle, rotPoint=None):
    (height,width) = img.shape[:2]
    
    if rotPoint is None:
        rotPoint = (width//2, height//2) #centre as the rotation point
       
    rotMat = cv.getRotationMatrix2D(rotPoint, angle, 0.85)
    dimensions = (width,height)
    
    return cv.warpAffine(img, rotMat, dimensions)


#trained resnet model
model = core.Model.load('model_weights_25.pth', ['Person']) #non augmented model


#funtion to compute the coordinates in the bird's eye view
def compute_point_perspective_transformation(matrix,boxes):
    list_downoids = [[box[0]+(box[2]-box[0])/2+300, box[3]+300] for box in boxes]
    #list_downoids = [[[box[0]+300, box[1]+300],[box[0]+box[2]+300,box[1]+300],[box[0]+300,box[1]+box[3]+300],[box[0]+box[2]+300,box[1]+box[3]+300]] for box in boxes]
    #list_downoids = [[box[4]+300, box[5]+box[3]//2+300] for box in boxes]
    list_points_to_detect = np.float32(list_downoids).reshape(-1, 1, 2)
    transformed_points = cv2.perspectiveTransform(list_points_to_detect, matrix)
    transformed_points_list = list()
    for i in range(0,transformed_points.shape[0]):
        transformed_points_list.append([transformed_points[i][0][0],transformed_points[i][0][1]])
    return np.array(transformed_points_list).astype('int')


#function to find euclidean distance between two points
def eucledian_distance(point1, point2):
    x1,y1 = point1
    x2,y2 = point2
    return sqrt((x1-x2)**2 + (y1-y2)**2)


#function to return a list of red and green boxes
def get_red_green_boxes(distance_allowed,birds_eye_points,boxes):
    red_boxes = []
    green_boxes = []

    new_boxes = [tuple(box) + tuple(result) for box, result in zip(boxes, birds_eye_points)]
    for i in range(0, len(new_boxes)-1):
            for j in range(i+1, len(new_boxes)):
                cxi,cyi = new_boxes[i][4:]
                cxj,cyj = new_boxes[j][4:]
                distance = eucledian_distance([cxi,cyi], [cxj,cyj])
                if distance < distance_allowed:
                    red_boxes.append(new_boxes[i])
                    red_boxes.append(new_boxes[j])

    green_boxes = list(set(new_boxes) - set(red_boxes))
    red_boxes = list(set(red_boxes))

    return (green_boxes, red_boxes)


#plot perspective view of violators (red) and non - violators (green)
def get_perspective_view_image(green_box, red_box,eye_view_height,eye_view_width):
    blank_image = cv2.imread('black_background.png')   
    for point in green_box:
        cv2.circle(blank_image,tuple([int(point[0]+(point[2]-point[0])/2),int(point[3])]),20,(0,255,0),-1)
    for point in red_box:
        cv2.circle(blank_image,tuple([int(point[0]+(point[2]-point[0])/2),int(point[3])]),20,(0,0,255),-1)
    blank_image = cv2.resize(blank_image,(eye_view_width,eye_view_height))
    return blank_image

#draw red rectangles around violators and green rectangle around non - violators
def get_red_green_box_image(new_box_image,green_box,red_box):
    for point in green_box:
        cv2.rectangle(new_box_image,(point[0],point[1]),(point[2],point[3]),(0, 255, 0), 2)
    for point in red_box:
        cv2.rectangle(new_box_image,(point[0],point[1]),(point[2],point[3]),(0, 0, 255), 2)
    return new_box_image


#getting points of red boxes
def red_boxes_points(boxes):
  lst = []
  for box in boxes:
    x = box[0] + (box[2] - box[0])/2 
    y = box[3] 
    a = (x,y)
    lst.append(a)
  return(lst)




Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
from moviepy.editor import *

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2768896/45929032 bytes (6.0%)6307840/45929032 bytes (13.7%)9781248/45929032 bytes (21.3%)13156352/45929032 bytes (28.6%)16015360/45929032 bytes (34.9%)19087360/45929032 bytes (41.6%)21667840/45929032 bytes (47.2%)24289280/45929032 bytes (52.9%)26656768/45929032 bytes (58.0%)28508160/45929032 bytes (62.1%)30343168/45929032 bytes (66.1%)32497664/45929032 bytes (70.8%)

In [ ]:
#reading the video file
video = cv2.VideoCapture('TownCentre.avi')
clip = VideoFileClip('OUTPUT_withoutAudio.avi')
final_vid = clip.subclip(0,1/25)
frame_number = 0
siren = 0

#Partition
person_near_store = [(790,406),(553,248),(0,488),(0,902)]
person_on_road_near_store = [(1335,472),(790,406),(0,902),(800,1060)]
person_near_black_pillar = [(1915,515),(1335,472),(800,1060),(1915,1079)]
person_near_entrance = [(1459,0),(1091,0),(553,248),(790,406)]
person_on_road_near_bench = [(1618,193),(1192,147),(790,406),(1335,472)]
person_near_sitting_place = [(1915,236),(1618,193),(1335,472),(1915,515)]
person_near_atm = [(1915,0),(1459,0),(1192,147),(1915,236)]

#point inside the polygon
pns = Polygon(person_near_store)
porns = Polygon(person_on_road_near_store)
pnbp = Polygon(person_near_black_pillar)
pne = Polygon(person_near_entrance)
pornb = Polygon(person_on_road_near_bench)
pnsp = Polygon(person_near_sitting_place)
pna = Polygon(person_near_atm)

#count of persons in each region
pns_count = 0
porns_count = 0
pnbp_count = 0
pne_count = 0
pornb_count = 0
pnsp_count = 0
pna_count = 0

#variables to note continuation of violation
pns_n = 0
porns_n = 0
pnbp_n = 0
pne_n = 0
pornb_n = 0
pnsp_n = 0
pna_n = 0

while True:
  
  #ret is a boolean variable that returns true if the frame is available and frame stores the image
  ret,frame = video.read()

  if not ret:
    break

  image_height, image_width = frame.shape[:2]    #image dimensions
  image = frame.copy()                           #copying the image

  img = padding(image,300)                       #padding

  # Coordinates that you want to Perspective Transform
  pts1 = np.float32([[1660,2],[2917,1184],[2,33],[482,2070]])
  # Size of the Transformed Image
  pts2 = np.float32([[0,0],[1920,0],[0,1080],[1920,1080]])
  #Warp Images
  H_matrix = cv2.getPerspectiveTransform(pts1,pts2)      #getting transformed matrix
  warped = cv2.warpPerspective(img,H_matrix,(1920,1080)) #getting warped image

  fin = rotate(warped,282, (warped.shape[:2][1]//1.6, warped.shape[:2][0]//2)) #rotating the warped image

  #predicting people in the image using model - resnet
  predictions = model.predict(image)
  labels, boxes, scores = predictions

  thresh=0.6 #set threshold for scores
  
  filtered_indices=np.where(scores>thresh)
  filtered_scores=scores[filtered_indices]
  filtered_boxes=boxes[filtered_indices]
  num_list = filtered_indices[0].tolist()
  filtered_labels = [labels[i] for i in num_list]
  boxes = filtered_boxes.numpy() #convert tensor to numpy

  birds_eye_points = compute_point_perspective_transformation(H_matrix, boxes) #getting the coordinates of people in warped image

  #reading each coordinates of people and drawing a circle at the bottom centre of people
  for i in range(0,len(birds_eye_points)):
    cv2.circle(warped,tuple([birds_eye_points[i][0],birds_eye_points[i][1]]), 15, (0, 0, 255), -1)

  min_distance = 122.12339 #it is the threshold pixel distance which people should maintain to avoid social distance violation

  green_box, red_box = get_red_green_boxes(min_distance, birds_eye_points, boxes) #getting the co-ordinates of the green box(non-violators) and red box(violators)
  
  #counting the frame number
  frame_number += 1
  print(frame_number)

  if frame_number >=1500: #the required number of output frames is mentioned here and the while loop breaks when the output video has reached the given number of frames
    break

  #intializing
  pns_count = 0
  porns_count = 0
  pnbp_count = 0
  pne_count = 0
  pornb_count = 0
  pnsp_count = 0
  pna_count = 0

  #demarcation
  red_point_box = red_boxes_points(red_box)
  #print(red_point_box)

  #check if the person is in a region
  for i in red_point_box:
    p1 = Point(i[0],i[1])
    if p1.within(pns) == True:
      pns_count +=1
      if pns_n == 0:
        pns_n = 1
    if p1.within(porns) == True:
      porns_count +=1
      if porns_n == 0:
        porns_n = 1
    if p1.within(pnbp) == True:
      pnbp_count +=1
      if pnbp_n == 0:
        pnbp_n = 1
    if p1.within(pne) == True:
      pne_count +=1
      if pne_n == 0:
        pne_n = 1
    if p1.within(pornb) == True:
      pornb_count +=1
      if pornb_n == 0:
        pornb_n = 1
    if p1.within(pnsp) == True:
      pnsp_count +=1
      if pnsp_n == 0:
        pnsp_n = 1
    if p1.within(pna) == True:
      pna_count +=1
      if pna_n == 0:
        pna_n = 1

  #to increase the count when the same red boxes continue in next frame
  if pns_count > 1 and pns_n !=0:
    pns_n += 1
  else:
    pns_n = 0
  if porns_count > 1 and porns_n !=0:
    porns_n += 1
  else:
    porns_n = 0
  if pnbp_count > 1 and pnbp_n !=0:
    pnbp_n += 1
  else:
    pnbp_n = 0
  if pne_count > 1 and pne_n !=0:
    pne_n += 1
  else:
    pne_n = 0
  if pornb_count > 1 and pornb_n !=0:
    pornb_n += 1
  else:
    pornb_n = 0
  if pnsp_count > 1 and pnsp_n !=0:
    pnsp_n += 1
  else:
    pnsp_n = 0
  if pna_count > 1 and pna_n !=0:
    pna_n += 1
  else:
    pna_n = 0
  
  #audio files should be inserted here
  if ((pns_n > 5*25) and (porns_n <= 2*25) and (pnbp_n <= 2*25) and (pne_n < 2*25) and (pornb_n <= 2*25) and (pnsp_n <= 2*25) and (pna_n <= 2*25)):
    audioclip = AudioFileClip("near_store_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    pns_n = 0
  elif ((pns_n <= 2*25) and (porns_n > 5*25) and (pnbp_n <= 2*25) and (pne_n <= 2*25) and (pornb_n <= 2*25) and (pnsp_n <= 2*25) and (pna_n <= 2*25)):
    audioclip = AudioFileClip("on_road_near_store_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    porns_n = 0
  elif ((pns_n <= 2*25) and (porns_n <= 2*25) and (pnbp_n > 5*25) and (pne_n <= 2*25) and (pornb_n <= 2*25) and (pnsp_n <= 2*25) and (pna_n <= 2*25)):
    audioclip = AudioFileClip("near_black_pillar_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    pnbp_n = 0
  elif ((pns_n <= 3*25) and (porns_n <= 3*25) and (pnbp_n <= 3*25) and (pne_n > 5*25) and (pornb_n <= 3*25) and (pnsp_n <= 3*25) and (pna_n <= 3*25)):
    audioclip = AudioFileClip("near_entrance_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    pne_n = 0
  elif ((pns_n <= 2*25) and (porns_n <= 2*25) and (pnbp_n <= 2*25) and (pne_n <= 2*25) and (pornb_n > 5*25) and (pnsp_n <= 2*25) and (pna_n <= 2*25)):
    audioclip = AudioFileClip("on_road_near_entrance_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    pornb_n = 0
  elif ((pns_n <= 2*25) and (porns_n <= 2*25) and (pnbp_n <= 2*25) and (pne_n <= 2*25) and (pornb_n <= 2*25) and (pnsp_n > 5*25) and (pna_n <= 2*25)):
    audioclip = AudioFileClip("near_sitting_place_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    pnsp_n = 0
  elif ((pns_n <= 2*25) and (porns_n <= 2*25) and (pnbp_n <= 2*25) and (pne_n <= 2*25) and (pornb_n <= 2*25) and (pnsp_n <= 2*25) and (pna_n > 5*25)):
    audioclip = AudioFileClip("near_atm_speed.mp3")
    videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
    videos = videoclip.set_audio(audioclip)
    final_vid = concatenate_videoclips([final_vid,videos])
    pna_n = 0
  elif ((pns_n <= 5*25) and (porns_n <= 5*25) and (pnbp_n <= 5*25) and (pne_n <= 5*25) and (pornb_n <= 5*25) and (pnsp_n <= 5*25) and (pna_n <= 5*25)):
    final = clip.subclip(frame_number/25,(frame_number+1)/25)
    final_vid = concatenate_videoclips([final_vid,final])
  else :
    audioclip = AudioFileClip("multiple_violations_speed.mp3")
    #once siren raised, it should play again only after 15 frames
    if siren < 15:
      siren += 1
      continue
    else:
      videoclip = clip.subclip(frame_number/25,(frame_number+1)/25)
      videos = videoclip.set_audio(audioclip)
      videos = videos.volumex(15)
      final_vid = concatenate_videoclips([final_vid,videos])
      siren = 0
    if pns_n >= 5*25:
      pns_n = 0
    if porns_n >= 5*25:
      porns_n = 0
    if pnbp_n >= 5*25:
      pnbp_n = 0
    if pne_n >= 5*25:
      pne_n = 0
    if pornb_n >= 5*25:
      pornb_n = 0
    if pnsp_n >= 5*25:
      pnsp_n = 0
    if pna_n >= 5*25:
      pna_n = 0

  del image,fin,warped #deleting the stored images so that new image can be loaded again
  
video.release()
final_vid.write_videofile('OUTPUT_withAudio_rev1_mv.mp4')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


100%|██████████| 727/727 [00:27<00:00, 26.14it/s] 

[MoviePy] Done.
[MoviePy] Writing video Trial_2.mp4



100%|█████████▉| 750/751 [01:23<00:00,  8.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Trial_2.mp4 



In [ ]:
img = plt.imread('frame0.jpg')
cv2.putText(img,'Store',(185,70),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
cv2.putText(img,'Entrance',(735,190),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
cv2.putText(img,'Road',(510,871),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
cv2.putText(img,'Bench',(569,437),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
cv2.putText(img,'Sitting Place',(1501,459),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
cv2.putText(img,'Black Pillar',(1192,829),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
cv2.putText(img,'ATM',(1659,70),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),4,2)
fig = plt.gcf()
DPI = fig.get_dpi()
fig.set_size_inches(1920/float(DPI),1080.0/float(DPI))
plt.imshow(img)
plt.show()